#Assumptions
1) created resuable generic functions in a common_functions notebook
2) created unit test cases for each function in unit_testing notebook
3) All the common functions will be tested using pytest using unit testing notebook before calling them in any data transformation notebooks
4) Used Pandas dataframe to read customers excel file maven excel library is unavailable in my free databricks serverless edition
5) created a catalog by name "orders_analysis" and schemas by name bronze,silver and gold under that catalog to store raw, enriched and aggregated tables
6) created a volume by name "landing" under bronze schema to store the source files


In [0]:
%pip install openpyxl

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from pyspark.sql.types import (
    StructType,
    StructField,
    LongType,
    IntegerType,
    StringType,
    DecimalType,
    DateType,
    TimestampType,
    DoubleType,
    DataType
)
from pyspark.sql.functions import (
    udf,
    to_date,
    col,
    to_timestamp,
    regexp_extract,
    regexp_replace,
    isnan,
    count,
    when,
    trim,
    initcap,length,lower,round,year,sum,ltrim
)
from pyspark.sql import SparkSession, DataFrame, Row
import re
import pandas as pd

# Create a SparkSession
spark = SparkSession.builder.appName("PEI_SD_Assessment").getOrCreate()

In [0]:
%run ./common_functions

In [0]:
spark.sql("CREATE CATALOG IF NOT EXISTS orders_analysis")
for schema in ["bronze", "silver", "gold"]:
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS orders_analysis.{schema}")

#Question:1

raw_orders

In [0]:
df_orders_raw = spark.read.option("multiline","true").json("/Volumes/orders_analysis/bronze/landing/Orders.json")
df_raw_orders_formatted = format_column_names(df_orders_raw)
create_and_verify_delta_table(
        spark=spark,
        df=df_raw_orders_formatted,
        catalog_name="orders_analysis",
        table_name="raw_orders",
        database="bronze",
        mode="overwrite",
        drop_if_exists=True,
        merge_schema=True
    )

True

In [0]:
%sql
select * from orders_analysis.bronze.raw_orders

Customer_ID Discount Order_Date Order_ID Price Product_ID Profit Quantity Row_ID Ship_Date Ship_Mode JK-15370 0.3 21/8/2016 CA-2016-122581 573.17 FUR-CH-10002961 63.69 7 1 25/8/2016 Standard Class BD-11320 0.0 23/9/2017 CA-2017-117485 291.96 TEC-AC-10004659 102.19 4 2 29/9/2017 Standard Class LB-16795 0.7 6/10/2016 US-2016-157490 17.0 OFF-BI-10002824 -14.92 4 3 7/10/2016 First Class KB-16315 0.2 2/7/2015 CA-2015-111703 15.55 OFF-PA-10003349 5.64 3 4 9/7/2015 Standard Class DO-13435 0.2 3/10/2014 CA-2014-108903 142.49 TEC-AC-10003023 -3.0 3 5 3/10/2014 Same Day CB-12025 0.0 27/11/2016 CA-2016-117583 79.95 OFF-BI-10004233 38.38 5 6 30/11/2016 First Class SM-20005 0.0 10/12/2014 CA-2014-148488 11.0 OFF-PA-10004470 5.23 2 7 15/12/2014 Standard Class RD-19480 0.0 1/12/2016 CA-2016-136434 17.31 FUR-FU-10001196 5.19 3 8 7/12/2016 Standard Class JM-16195 0.0 30/4/2014 CA-2014-160094 826.0 OFF-ST-10000585 214.0 5 9 2/5/2014 First Class SC-20230 0.0 3/8/2017 CA-2017-141747 16.06 OFF-ST-10003996 4.18 1 10 8/8/2017 Second Class BO-11350 0.2 3/5/2017 CA-2017-132199 8.0 OFF-FA-10002280 2.8 2 11 8/5/2017 Standard Class BD-11320 0.2 27/11/2017 CA-2017-107125 117.49 OFF-BI-10001989 41.12 7 12 2/12/2017 Standard Class AB-10105 0.7 19/9/2017 CA-2017-153822 18.18 OFF-BI-10001460 -13.94 4 13 25/9/2017 Standard Class NP-18670 0.0 12/10/2017 CA-2017-150091 45.0 TEC-AC-10002167 4.0 3 14 16/10/2017 Standard Class KD-16270 0.0 2/9/2016 CA-2016-130407 39.98 FUR-FU-10001967 9.99 2 15 6/9/2016 Standard Class BF-11005 0.6 28/7/2016 US-2016-105452 302.0 FUR-FU-10003806 -378.4 5 16 1/8/2016 Standard Class LE-16810 0.8 27/5/2014 US-2014-117058 17.46 OFF-BI-10004139 -30.55 6 17 30/5/2014 First Class TT-21070 0.0 13/11/2017 CA-2017-122490 344.91 OFF-ST-10000991 10.35 3 18 18/11/2017 Standard Class CA-12055 0.2 22/11/2016 US-2016-164945 134.27 OFF-BI-10001524 47.0 8 19 27/11/2016 Standard Class GD-14590 0.0 5/5/2014 CA-2014-111934 11.88 OFF-BI-10004364 5.0 2 20 7/5/2014 First Class DK-12895 0.0 8/3/2015 CA-2015-105627 373.08 FUR-FU-10000308 82.08 6 21 12/3/2015 Standard Class RM-19675 0.0 19/11/2014 CA-2014-158274 503.96 TEC-PH-10003273 131.03 4 22 24/11/2014 Second Class MY-18295 0.0 28/4/2014 CA-2014-118976 5.34 OFF-AR-10004752 1.5 2 23 3/5/2014 Standard Class DB-13555 0.2 14/9/2017 CA-2017-115882 74.35 OFF-PA-10001125 23.23 3 24 17/9/2017 First Class JS-15880 0.0 21/12/2015 US-2015-126214 1618.0 FUR-TA-10003748 356.04 13 25 24/12/2015 Second Class MJ-17740 0.0 11/3/2017 CA-2017-147291 895.92 OFF-BI-10003091 421.08 4 26 17/3/2017 Standard Class JL-15835 0.0 17/12/2015 US-2015-138121 142.36 FUR-CH-10004875 38.44 2 27 17/12/2015 Same Day PF-19120 0.8 7/4/2015 CA-2015-140025 463.25 OFF-AP-10002651 -1181.28 8 28 11/4/2015 Standard Class KB-16315 0.0 26/8/2014 CA-2014-103331 3.38 OFF-PA-10002659 1.55 1 29 1/9/2014 Standard Class DO-13645 0.0 4/12/2017 CA-2017-118003 302.94 FUR-CH-10004860 18.18 3 30 10/12/2017 Standard Class PG-18820 0.0 27/7/2015 US-2015-129553 29.97 TEC-PH-10004897 0.3 3 31 2/8/2015 Standard Class AJ-10780 0.0 14/5/2017 US-2017-133361 14.0 OFF-AR-10003045 6.62 5 32 17/5/2017 First Class JG-15115 0.7 3/12/2015 CA-2015-123092 95.0 OFF-BI-10001718 -73.58 5 33 7/12/2015 Standard Class SC-20380 0.2 3/2/2017 CA-2017-126354 115.3 OFF-PA-10004381 40.35 3 34 8/2/2017 Standard Class KH-16630 0.0 18/9/2015 CA-2015-109575 499.99 TEC-PH-10000730 130.0 1 35 23/9/2015 Standard Class KL-16555 0.7 26/12/2014 CA-2014-111451 78.6 OFF-BI-10004593 -62.88 5 36 28/12/2014 First Class TH-21550 0.0 26/6/2017 CA-2017-133256 543.92 TEC-PH-10002660 135.98 8 37 27/6/2017 First Class PB-19105 0.2 15/11/2014 CA-2014-128888 604.66 OFF-EN-10003001 204.07 9 38 22/11/2014 Standard Class GM-14680 0.2 30/9/2016 CA-2016-106243 523.92 FUR-BO-10003034 -26.2 5 39 4/10/2016 Standard Class FH-14275 0.2 3/7/2017 CA-2017-121503 273.9 OFF-PA-10001878 92.44 7 40 6/7/2017 Second Class JP-15460 0.8 23/11/2017 CA-2017-156622 6.1 OFF-BI-10003707 -9.16 2 41 26/11/2017 First Class JD-16015 0.8 14/1

raw_customers

In [0]:
# Read the Excel file into a pandas DataFrame since maven execl package is not available for my testing
pd_customer = pd.read_excel("/Volumes/orders_analysis/bronze/landing/Customer.xlsx")

# Ensure the 'phone' column is of type string
pd_customer["phone"] = pd_customer["phone"].astype(str)

# Create a Spark DataFrame from the pandas DataFrame
spark_df = spark.createDataFrame(pd_customer)

df_raw_customers = format_column_names(spark_df)
create_and_verify_delta_table(
    spark=spark,
    df=df_raw_customers,
    catalog_name="orders_analysis",
    table_name="raw_customers",
    database="bronze",
    mode="overwrite",
    drop_if_exists=True,
    merge_schema=True,
)

True

In [0]:
%sql
select * from orders_analysis.bronze.raw_customers

Customer_ID Customer_Name email phone address Segment Country City State Postal_Code Region PW-19240 Pierre Wener bettysullivan808@gmail.com 421.580.0902x9815 001 Jones Ridges Suite 338
Johnsonfort, FL 95462 Consumer United States Louisville Colorado 80027 West GH-14410 Gary567 Hansen austindyer948@gmail.com 001-542-415-0246x314 00347 Murphy Unions
Ashleyton, IA 29814 Home Office United States Chicago Illinois 60653 Central KL-16555 Kelly Lampkin clarencehughes280@gmail.com 7185624866 007 Adams Lane Suite 176
East Amyberg, IN 34581 Corporate United States Colorado Springs Colorado 80906 West AH-10075 Ad. ..am Hart angelabryant256@gmail.com 265.101.5569x1098 01454 Christopher Turnpike
North Ryanstad, MI 36226 Corporate United States Columbus Ohio 43229 East PF-19165 Philip Fox kristinereynolds576@gmail.com 001-473-645-2141x9154 0158 Harris Ways Suite 085
East Laceyside, SD 35649 Consumer United States San Diego California 92105 West SC-20680 Steve Carroll jasoncontreras178@gmail.com (563)647-4830x5318 01630 Tammy Prairie
North Daniel, KS 26404 Home Office United States Seattle Washington 98105 West JR-15700 Jocasta Rupert johncombs689@gmail.com -6181 019 Emily Corner Apt. 810
Ryantown, SC 37010 Consumer United States Jacksonville Florida 32216 South AB-10105 Adrian Barton daviddavis980@gmail.com 067.435.8553x692 021 Katherine Mall
Jameston, DC 24685 Consumer United States Phoenix Arizona 85023 West PT-19090 Pete@#$ Takahito mikaylaarnold666@gmail.com 786.638.6820 0236 Lane Squares
Port Samantha, ME 15670 Consumer United States San Antonio Texas 78207 Central SG-20605 Speros Goranitis brianjoyce110@gmail.com 3528465094 02401 Angela Loop Apt. 678
Port John, ME 43448 Consumer United States Lafayette Indiana 47905 Central MH-17785 Maya Herman christinasalas345@gmail.com (722)376-5599x0693 026 Colon Hill
New Pedromouth, CA 18437 Corporate United States San Diego California 92105 West KB-16240 Karen Bern christopherperez199@gmail.com 817.409.0760x92195 026 White Squares
Robertton, VA 78741 Corporate United States Odessa Texas 79762 Central JM-15535 Jessica Myrick nicholasrussell869@gmail.com 924.484.7935 02785 Johnson Shore
South Nathan, KY 45544 Consumer United States New York City New York 10035 East JH-16180 Justin Hirsh kellymartinez374@gmail.com 001-361-720-6500x0984 0317 Parker Lane
Port Jennifer, OK 19258 Consumer United States Philadelphia Pennsylvania 19140 East CS-12400 Christopher Schild matthewwilliams637@gmail.com 106-229-5443x7580 034 Lynch Squares
North Benjaminside, MH 42764 Home Office United States Philadelphia Pennsylvania 19134 East LR-17035 Lisa Ryan lorimorrow317@gmail.com (549)707-5809 035 Cox View Suite 514
East Carlton, VA 31206 Corporate United States Santa Clara California 95051 West TW-21025 Tam&^*ara Willing___)ham selenajones479@gmail.com 2323700362 03551 Michael Shore
Lake Barryview, MH 82312 Home Office United States Charlottesville Virginia 22901 South SR-20425 Sharelle Roach raymondpeck288@gmail.com 897-362-7773 03958 Shane Lakes
New Jessicatown, AR 78525 Home Office United States Louisville Colorado 80027 West AS-10285 Alejandro Savely laceymercado410@gmail.com 2167104605 04225 Lee Manor
Castroville, HI 72528 Corporate United States San Francisco California 94109 West KB-16405 Kat rina Bavinger michaelwood756@gmail.com 001-347-866-9161x248 0432 Jennifer Port Suite 897
Johnstad, AR 12201 Home Office United States Apple Valley California 92307 West DW-13480 Dianna Wilson alejandrobailey724@gmail.com (432)851-1371x262 04653 Fischer Valley
East Daltonhaven, MO 31850 Home Office United States Lakeville Minnesota 55044 Central JG-15310 Jason Gross danielpaul555@gmail.com 035.841.4918x9187 04852 Wise Row
Davidfurt, AR 32041 Corporate United States Newark Ohio 43055 East TS-21085 Thais Sissman jamesjohnson660@gmail.com 001-295-152-2880x529 04890 Sweeney Turnpike
East Mariaside, DC 09754 Consumer United States Ormond Beach Florida 32174 South KN-16705 Kristi;'[]na Nunn sabrinahayes269@gmail.com #ERROR! 04950

raw_products

In [0]:
df_raw_products = spark.read.csv(
    "/Volumes/orders_analysis/bronze/landing/Products.csv",
    header=True,
    quote='"',  # Use " as the quote character
    escape='"',  # Use " as the escape character for quotes inside fields
    multiLine=True,  # Optional, if values span multiple lines
)
df_raw_products = format_column_names(df_raw_products)
create_and_verify_delta_table(
    spark=spark,
    df=df_raw_products,
    catalog_name="orders_analysis",
    table_name="raw_products",
    database="bronze",
    mode="overwrite",
    drop_if_exists=True,
    merge_schema=True,
)

True

In [0]:
%sql
select * from orders_analysis.bronze.raw_products

Product_ID Category Sub_Category Product_Name State Price_per_product FUR-CH-10002961 Furniture Chairs Leather Task Chair, Black New York 81.882 TEC-AC-10004659 Technology Accessories Imation Secure+ Hardware Encrypted USB 2.0 Flash Drive; 16GB Oklahoma 72.99 OFF-BI-10002824 Office Supplies Binders Recycled Easel Ring Binders Colorado 4.25 OFF-PA-10003349 Office Supplies Paper Xerox 1957 Florida 5.184 TEC-AC-10003023 Technology Accessories Logitech G105 Gaming Keyboard Ohio 47.496 OFF-BI-10004233 Office Supplies Binders GBC Pre-Punched Binding Paper, Plastic, White, 8-1/2" x 11" New Jersey 15.99 OFF-PA-10004470 Office Supplies Paper Adams Write n' Stick Phone Message Book, 11" X 5 1/4", 200 Messages New York 5.5 FUR-FU-10001196 Furniture Furnishings DAX Cubicle Frames - 8x10 Indiana 5.77 OFF-ST-10000585 Office Supplies Storage Economy Rollaway Files Kentucky 165.2 OFF-ST-10003996 Office Supplies Storage Letter/Legal File Tote with Clear Snap-On Lid, Black Granite Washington 16.06 OFF-FA-10002280 Office Supplies Fasteners Advantus Plastic Paper Clips Pennsylvania 4 OFF-BI-10001989 Office Supplies Binders Premium Transparent Presentation Covers by GBC California 16.784 OFF-BI-10001460 Office Supplies Binders Plastic Binding Combs Arizona 4.545 TEC-AC-10002167 Technology Accessories Imation 8gb Micro Traveldrive Usb 2.0 Flash Drive New Jersey 15 FUR-FU-10001967 Furniture Furnishings Telescoping Adjustable Floor Lamp New York 19.99 FUR-FU-10003806 Furniture Furnishings Tenex Chairmat w/ Average Lip, 45" x 53" Texas 60.4 OFF-BI-10004139 Office Supplies Binders Fellowes Presentation Covers for Comb Binding Machines Illinois 2.91 OFF-ST-10000991 Office Supplies Storage Space Solutions HD Industrial Steel Shelving. Washington 114.97 OFF-BI-10001524 Office Supplies Binders GBC Premium Transparent Covers with Diagonal Lined Pattern New York 16.784 OFF-BI-10004364 Office Supplies Binders Storex Dura Pro Binders Virginia 5.94 FUR-FU-10000308 Furniture Furnishings Deflect-o Glass Clear Studded Chair Mats Wisconsin 62.18 TEC-PH-10003273 Technology Phones AT&T TR1909W Louisiana 125.99 OFF-AR-10004752 Office Supplies Art Blackstonian Pencils Maryland 2.67 OFF-PA-10001125 Office Supplies Paper Xerox 1988 North Carolina 24.784 FUR-TA-10003748 Furniture Tables Bevis 36 x 72 Conference Tables Washington 124.4615385 OFF-BI-10003091 Office Supplies Binders GBC DocuBind TL200 Manual Binding Machine Michigan 223.98 FUR-CH-10004875 Furniture Chairs Harbour Creations 67200 Series Stacking Chairs Michigan 71.18 OFF-AP-10002651 Office Supplies Appliances Hoover Upright Vacuum With Dirt Cup Texas 57.906 OFF-PA-10002659 Office Supplies Paper Avoid Verbal Orders Carbonless Minifold Book Delaware 3.38 FUR-CH-10004860 Furniture Chairs Global Low Back Tilter Chair New Jersey 100.98 TEC-PH-10004897 Technology Phones Mediabridge Sport Armband iPhone 5s Kentucky 9.99 OFF-AR-10003045 Office Supplies Art Prang Colored Pencils Maryland 2.8 OFF-BI-10001718 Office Supplies Binders GBC DocuBind P50 Personal Binding Machine North Carolina 19 OFF-PA-10004381 Office Supplies Paper 14-7/8 x 11 Blue Bar Computer Printout Paper Florida 38.432 TEC-PH-10000730 Technology Phones Samsung Galaxy S4 Active Maryland 499.99 OFF-BI-10004593 Office Supplies Binders Ibico Laser Imprintable Binding System Covers Colorado 15.72 TEC-PH-10002660 Technology Phones Nortel Networks T7316 E Nt8 B27 Michigan 67.99 OFF-EN-10003001 Office Supplies Envelopes Ames Color-File Green Diamond Border X-ray Mailers Texas 67.184 FUR-BO-10003034 Furniture Bookcases O'Sullivan Elevations Bookcase, Cherry Finish New York 104.784 OFF-PA-10001878 Office Supplies Paper Xerox 1891 Texas 39.128 OFF-BI-10003707 Office Supplies Binders Aluminum Screw Posts Texas 3.052 OFF-BI-10001249 Office Supplies Binders Avery Heavy-Duty EZD View Binder with Locking Rings Illinois 1.276 FUR-TA-10001539 Furniture Tables Chromcraft Rectangular Conference Tables California 189.576 FUR-BO-10001972 Furniture Bookcases O'Sullivan 4-Sh

#Question:2

**Enriched Customers**

In [0]:
df_customers = spark.table("orders_analysis.bronze.raw_customers")
customer_data_type_map = {
    "Customer_ID": StringType(),
    "Customer_Name": StringType(),
    "email": StringType(),
    "phone": StringType(),
    "address": StringType(),
    "Segment": StringType(),
    "Country": StringType(),
    "City": StringType(),
    "State": StringType(),
    "Postal_Code": IntegerType(),
    "Region": StringType(),
}

df_customers = clean_name(df_customers, "Customer_Name")
df_customers = clean_phone_numbers(df_customers, "phone")
df_customers = clean_email_column(df_customers, "email")
# Apply casting
df_enriched_customers = cast_columns(df_customers, customer_data_type_map)

create_and_verify_delta_table(
    spark=spark,
    df=df_enriched_customers,
    catalog_name="orders_analysis",
    table_name="customers",
    database="silver",
    mode="overwrite",
    drop_if_exists=True,
    merge_schema=True,
)

True

In [0]:
%sql
select * from orders_analysis.silver.customers

Customer_ID Customer_Name email phone address Segment Country City State Postal_Code Region PW-19240 Pierre Wener bettysullivan808@gmail.com 4215800902 001 Jones Ridges Suite 338
Johnsonfort, FL 95462 Consumer United States Louisville Colorado 80027 West GH-14410 Gary Hansen austindyer948@gmail.com 5424150246 00347 Murphy Unions
Ashleyton, IA 29814 Home Office United States Chicago Illinois 60653 Central KL-16555 Kelly Lampkin clarencehughes280@gmail.com 7185624866 007 Adams Lane Suite 176
East Amyberg, IN 34581 Corporate United States Colorado Springs Colorado 80906 West AH-10075 Ad Am Hart angelabryant256@gmail.com 2651015569 01454 Christopher Turnpike
North Ryanstad, MI 36226 Corporate United States Columbus Ohio 43229 East PF-19165 Philip Fox kristinereynolds576@gmail.com 4736452141 0158 Harris Ways Suite 085
East Laceyside, SD 35649 Consumer United States San Diego California 92105 West SC-20680 Steve Carroll jasoncontreras178@gmail.com 5636474830 01630 Tammy Prairie
North Daniel, KS 26404 Home Office United States Seattle Washington 98105 West JR-15700 Jocasta Rupert johncombs689@gmail.com null 019 Emily Corner Apt. 810
Ryantown, SC 37010 Consumer United States Jacksonville Florida 32216 South AB-10105 Adrian Barton daviddavis980@gmail.com 0674358553 021 Katherine Mall
Jameston, DC 24685 Consumer United States Phoenix Arizona 85023 West PT-19090 Pete Takahito mikaylaarnold666@gmail.com 7866386820 0236 Lane Squares
Port Samantha, ME 15670 Consumer United States San Antonio Texas 78207 Central SG-20605 Speros Goranitis brianjoyce110@gmail.com 3528465094 02401 Angela Loop Apt. 678
Port John, ME 43448 Consumer United States Lafayette Indiana 47905 Central MH-17785 Maya Herman christinasalas345@gmail.com 7223765599 026 Colon Hill
New Pedromouth, CA 18437 Corporate United States San Diego California 92105 West KB-16240 Karen Bern christopherperez199@gmail.com 8174090760 026 White Squares
Robertton, VA 78741 Corporate United States Odessa Texas 79762 Central JM-15535 Jessica Myrick nicholasrussell869@gmail.com 9244847935 02785 Johnson Shore
South Nathan, KY 45544 Consumer United States New York City New York 10035 East JH-16180 Justin Hirsh kellymartinez374@gmail.com 3617206500 0317 Parker Lane
Port Jennifer, OK 19258 Consumer United States Philadelphia Pennsylvania 19140 East CS-12400 Christopher Schild matthewwilliams637@gmail.com 1062295443 034 Lynch Squares
North Benjaminside, MH 42764 Home Office United States Philadelphia Pennsylvania 19134 East LR-17035 Lisa Ryan lorimorrow317@gmail.com 5497075809 035 Cox View Suite 514
East Carlton, VA 31206 Corporate United States Santa Clara California 95051 West TW-21025 Tamara Willingham selenajones479@gmail.com 2323700362 03551 Michael Shore
Lake Barryview, MH 82312 Home Office United States Charlottesville Virginia 22901 South SR-20425 Sharelle Roach raymondpeck288@gmail.com 8973627773 03958 Shane Lakes
New Jessicatown, AR 78525 Home Office United States Louisville Colorado 80027 West AS-10285 Alejandro Savely laceymercado410@gmail.com 2167104605 04225 Lee Manor
Castroville, HI 72528 Corporate United States San Francisco California 94109 West KB-16405 Kat Rina Bavinger michaelwood756@gmail.com 3478669161 0432 Jennifer Port Suite 897
Johnstad, AR 12201 Home Office United States Apple Valley California 92307 West DW-13480 Dianna Wilson alejandrobailey724@gmail.com 4328511371 04653 Fischer Valley
East Daltonhaven, MO 31850 Home Office United States Lakeville Minnesota 55044 Central JG-15310 Jason Gross danielpaul555@gmail.com 0358414918 04852 Wise Row
Davidfurt, AR 32041 Corporate United States Newark Ohio 43055 East TS-21085 Thais Sissman jamesjohnson660@gmail.com 2951522880 04890 Sweeney Turnpike
East Mariaside, DC 09754 Consumer United States Ormond Beach Florida 32174 South KN-16705 Kristina Nunn sabrinahayes269@gmail.com null 04950 Joseph Meadow
East Kyleville, FM 82237 Home Office United States Sparks Nevada 89431 West CM-12115 Chad Mcguire sharonwarner980@gmail.com 0339346095 

**Enriched Products**

In [0]:
df_products = spark.table("orders_analysis.bronze.raw_products")
product_data_type_map = {
    "Product_ID": StringType(),
    "Category": StringType(),
    "Sub_Category": StringType(),
    "Product_Name": StringType(),
    "State": StringType(),
    "Price_per_product": DoubleType(),
}

df_products = orders_extract_double(
    df_products, "Price_per_product", "Price_per_product"
)
# Apply casting
df_enriched_products = cast_columns(df_products, product_data_type_map)
create_and_verify_delta_table(
    spark=spark,
    df=df_enriched_products,
    catalog_name="orders_analysis",
    table_name="products",
    database="silver",
    mode="overwrite",
    drop_if_exists=True,
    merge_schema=True,
)

True

In [0]:
%sql
select * from orders_analysis.silver.products

Product_ID Category Sub_Category Product_Name State Price_per_product FUR-CH-10002961 Furniture Chairs Leather Task Chair, Black New York 81.882 TEC-AC-10004659 Technology Accessories Imation Secure+ Hardware Encrypted USB 2.0 Flash Drive; 16GB Oklahoma 72.99 OFF-BI-10002824 Office Supplies Binders Recycled Easel Ring Binders Colorado 4.25 OFF-PA-10003349 Office Supplies Paper Xerox 1957 Florida 5.184 TEC-AC-10003023 Technology Accessories Logitech G105 Gaming Keyboard Ohio 47.496 OFF-BI-10004233 Office Supplies Binders GBC Pre-Punched Binding Paper, Plastic, White, 8-1/2" x 11" New Jersey 15.99 OFF-PA-10004470 Office Supplies Paper Adams Write n' Stick Phone Message Book, 11" X 5 1/4", 200 Messages New York 5.5 FUR-FU-10001196 Furniture Furnishings DAX Cubicle Frames - 8x10 Indiana 5.77 OFF-ST-10000585 Office Supplies Storage Economy Rollaway Files Kentucky 165.2 OFF-ST-10003996 Office Supplies Storage Letter/Legal File Tote with Clear Snap-On Lid, Black Granite Washington 16.06 OFF-FA-10002280 Office Supplies Fasteners Advantus Plastic Paper Clips Pennsylvania 4.0 OFF-BI-10001989 Office Supplies Binders Premium Transparent Presentation Covers by GBC California 16.784 OFF-BI-10001460 Office Supplies Binders Plastic Binding Combs Arizona 4.545 TEC-AC-10002167 Technology Accessories Imation 8gb Micro Traveldrive Usb 2.0 Flash Drive New Jersey 15.0 FUR-FU-10001967 Furniture Furnishings Telescoping Adjustable Floor Lamp New York 19.99 FUR-FU-10003806 Furniture Furnishings Tenex Chairmat w/ Average Lip, 45" x 53" Texas 60.4 OFF-BI-10004139 Office Supplies Binders Fellowes Presentation Covers for Comb Binding Machines Illinois 2.91 OFF-ST-10000991 Office Supplies Storage Space Solutions HD Industrial Steel Shelving. Washington 114.97 OFF-BI-10001524 Office Supplies Binders GBC Premium Transparent Covers with Diagonal Lined Pattern New York 16.784 OFF-BI-10004364 Office Supplies Binders Storex Dura Pro Binders Virginia 5.94 FUR-FU-10000308 Furniture Furnishings Deflect-o Glass Clear Studded Chair Mats Wisconsin 62.18 TEC-PH-10003273 Technology Phones AT&T TR1909W Louisiana 125.99 OFF-AR-10004752 Office Supplies Art Blackstonian Pencils Maryland 2.67 OFF-PA-10001125 Office Supplies Paper Xerox 1988 North Carolina 24.784 FUR-TA-10003748 Furniture Tables Bevis 36 x 72 Conference Tables Washington 124.4615385 OFF-BI-10003091 Office Supplies Binders GBC DocuBind TL200 Manual Binding Machine Michigan 223.98 FUR-CH-10004875 Furniture Chairs Harbour Creations 67200 Series Stacking Chairs Michigan 71.18 OFF-AP-10002651 Office Supplies Appliances Hoover Upright Vacuum With Dirt Cup Texas 57.906 OFF-PA-10002659 Office Supplies Paper Avoid Verbal Orders Carbonless Minifold Book Delaware 3.38 FUR-CH-10004860 Furniture Chairs Global Low Back Tilter Chair New Jersey 100.98 TEC-PH-10004897 Technology Phones Mediabridge Sport Armband iPhone 5s Kentucky 9.99 OFF-AR-10003045 Office Supplies Art Prang Colored Pencils Maryland 2.8 OFF-BI-10001718 Office Supplies Binders GBC DocuBind P50 Personal Binding Machine North Carolina 19.0 OFF-PA-10004381 Office Supplies Paper 14-7/8 x 11 Blue Bar Computer Printout Paper Florida 38.432 TEC-PH-10000730 Technology Phones Samsung Galaxy S4 Active Maryland 499.99 OFF-BI-10004593 Office Supplies Binders Ibico Laser Imprintable Binding System Covers Colorado 15.72 TEC-PH-10002660 Technology Phones Nortel Networks T7316 E Nt8 B27 Michigan 67.99 OFF-EN-10003001 Office Supplies Envelopes Ames Color-File Green Diamond Border X-ray Mailers Texas 67.184 FUR-BO-10003034 Furniture Bookcases O'Sullivan Elevations Bookcase, Cherry Finish New York 104.784 OFF-PA-10001878 Office Supplies Paper Xerox 1891 Texas 39.128 OFF-BI-10003707 Office Supplies Binders Aluminum Screw Posts Texas 3.052 OFF-BI-10001249 Office Supplies Binders Avery Heavy-Duty EZD View Binder with Locking Rings Illinois 1.276 FUR-TA-10001539 Furniture Tables Chromcraft Rectangular Conference Tables California 189.576 FUR-BO-10001972 Furniture Bookcases O'Sulliva

#Question:3

In [0]:
df_raw_orders = spark.table("orders_analysis.bronze.raw_orders")
df_customers = spark.table("orders_analysis.silver.customers")
df_products = spark.table("orders_analysis.silver.products")
# Define orders data types
orders_data_type_map = {
    "Row_ID": IntegerType(),
    "Order_ID": StringType(),
    "Order_Date": DateType(),
    "Ship_Date": DateType(),
    "Ship_Mode": StringType(),
    "Customer_ID": StringType(),
    "Product_ID": StringType(),
    "Quantity": IntegerType(),
    "Price": DoubleType(),
    "Discount": DoubleType(),
    "Profit": DoubleType(),
}

# Optional date format overrides
date_format_map = {"Order Date": "dd/MM/yyyy", "Ship Date": "dd/MM/yyyy"}

df_order_price_extract = orders_extract_double(df_raw_orders, "price", "price")

# Apply casting
df_casted_orders = cast_columns(
    df_order_price_extract, orders_data_type_map, date_format_map
)
# Order Enrichment
df_enriched_orders = (
    df_casted_orders.alias("o")
    .join(
        df_customers.alias("c"),
        df_casted_orders["Customer_ID"] == df_customers["Customer_ID"],
    )
    .join(
        df_products.alias("p"),
        df_casted_orders["Product_ID"] == df_products["Product_ID"],
    )
    .select(
        df_casted_orders["*"],
        df_customers["Customer_Name"],
        df_customers["Country"],
        df_products["Category"],
        df_products["Sub_Category"],
    )
    .withColumn("Profit", round(col("Profit"), 2))
)
create_and_verify_delta_table(
    spark=spark,
    df=df_enriched_orders,
    catalog_name="orders_analysis",
    table_name="orders",
    database="silver",
    mode="overwrite",
    drop_if_exists=True,
    merge_schema=True,
)

True

#Question:4

In [0]:
df_orders = spark.table("orders_analysis.silver.orders")
df_profit_by_year_pc_psc_customer = (
    df_orders.groupBy(
        year(col("Order_Date")).alias("Year"),
        col("Category"),
        col("Sub_Category"),
        col("Customer_ID"),
        col("Customer_Name"),
    )
    .agg(sum("Profit").alias("Total_Profit"))
    .withColumn("Total_Profit", round(col("Total_Profit"), 2))
    .orderBy(col("Year").desc())
)

create_and_verify_delta_table(
    spark=spark,
    df=df_profit_by_year_pc_psc_customer,
    catalog_name="orders_analysis",
    table_name="profit_aggregated",
    database="gold",
    mode="overwrite",
    drop_if_exists=True,
    merge_schema=True,
)

True

#Question:5a

In [0]:
%sql
Select
  Year,
  round(sum(Total_Profit), 2) as Profit_By_Year
from
  orders_analysis.gold.profit_aggregated
group by
  Year
order by
  Year desc

Year Profit_By_Year 2017 127175.23 2016 68161.13 2015 65706.25 2014 40975.4

#Question:5b

In [0]:
%sql
Select
  Year,
  Category,
  round(sum(Total_Profit), 2) as Profit_By_Year_Category
from
  orders_analysis.gold.profit_aggregated
group by
  Year,
  Category
order by
  Year,
  Category desc

Year Category Profit_By_Year_Category 2014 Technology 23486.29 2014 Office Supplies 22663.76 2014 Furniture -5174.65 2015 Technology 36823.79 2015 Office Supplies 25490.34 2015 Furniture 3392.12 2016 Technology 24437.38 2016 Office Supplies 35973.6 2016 Furniture 7750.15 2017 Technology 78483.08 2017 Office Supplies 45330.39 2017 Furniture 3361.76

#Question:5c

In [0]:
%sql
Select
  customer_id,
  round(sum(Total_Profit), 2) as Profit_By_customer_id
from
  orders_analysis.gold.profit_aggregated
group by
  customer_id
order by
  customer_id desc

customer_id Profit_By_customer_id ZD-21925 252.09 ZC-21910 -947.96 YS-21880 1778.31 YC-21895 1305.64 XP-21865 630.7 WB-21850 714.52 VW-21775 -874.66 VT-21700 -224.93 VS-21820 -317.08 VP-21760 -1018.91 VP-21730 205.39 VM-21835 107.33 VM-21685 513.62 VG-21805 -52.33 VG-21790 144.12 VF-21715 -93.1 VD-21670 1676.61 VB-21745 371.25 TZ-21580 233.92 TW-21025 737.39 TT-21460 1056.66 TT-21265 93.92 TT-21220 283.22 TT-21070 771.21 TS-21655 220.73 TS-21610 -315.47 TS-21505 11.39 TS-21430 48.9 TS-21370 2369.48 TS-21340 -165.38 TS-21205 853.59 TS-21160 266.85 TS-21085 -3.32 TR-21325 220.36 TP-21565 139.44 TP-21415 -1022.86 TP-21130 1128.65 TN-21040 -692.05 TM-21490 292.53 TM-21010 589.71 TH-21550 -438.84 TH-21235 83.53 TH-21115 -252.55 TH-21100 -135.22 TG-21640 365.12 TG-21310 682.17 TD-20995 74.84 TC-21535 217.95 TC-21475 119.13 TC-21295 686.27 TC-21145 265.59 TC-20980 8983.23 TB-21625 379.8 TB-21595 -136.47 TB-21520 -1635.35 TB-21400 2821.71 TB-21355 268.95 TB-21280 735.74 TB-21250 260.29 TB-21190 179.01 TB-21175 828.71 TB-21055 384.5 TA-21385 4703.72 SZ-20035 366.42 SW-20755 68.23 SW-20455 793.64 SW-20350 95.84 SW-20275 332.76 SW-20245 -19.58 SV-20935 93.84 SV-20815 199.64 SV-20785 484.85 SV-20365 1200.14 SU-20665 763.7 ST-20530 132.18 SS-20875 -65.35 SS-20590 135.91 SS-20515 210.98 SS-20410 -1398.04 SS-20140 332.03 SR-20740 2006.67 SR-20425 -3334.17 SP-20920 10.91 SP-20860 584.42 SP-20650 215.28 SP-20620 -656.58 SP-20545 -118.38 SO-20335 -26.3 SN-20710 554.7 SN-20560 -536.07 SM-20950 581.57 SM-20905 -16475.55 SM-20320 -1978.12 SM-20005 -35.39 SL-20155 528.04 SK-19990 168.79 SJ-20500 68.59 SJ-20215 -24.22 SJ-20125 143.72 SH-20635 260.63 SH-20395 -142.28 SH-19975 548.63 SG-20890 -10.33 SG-20605 463.27 SG-20470 -133.21 SG-20080 590.48 SF-20965 539.94 SF-20200 270.61 SF-20065 227.76 SE-20110 2661.23 SD-20485 1985.18 SC-20845 31.17 SC-20800 52.21 SC-20770 -1272.94 SC-20725 1164.69 SC-20695 611.82 SC-20680 360.97 SC-20575 100.23 SC-20440 379.57 SC-20380 -3189.0 SC-20305 264.44 SC-20260 670.98 SC-20230 439.82 SC-20095 5839.14 SC-20050 1399.94 SC-20020 -298.59 SB-20290 -2082.76 SB-20185 880.95 SB-20170 157.68 SA-20830 596.78 RW-19690 707.55 RW-19630 699.06 RW-19540 1574.55 RS-19870 9.58 RS-19765 1206.39 RS-19420 633.45 RR-19525 721.69 RR-19315 -73.83 RP-19855 586.92 RP-19390 469.96 RP-19270 59.52 RO-19780 -1262.81 RM-19750 28.68 RM-19675 1902.55 RM-19375 398.36 RL-19615 213.98 RK-19300 273.79 RH-19600 176.39 RH-19555 0.31 RH-19510 1370.53 RH-19495 563.78 RF-19840 402.85 RF-19735 711.83 RF-19345 633.67 RE-19450 209.25 RE-19405 6.04 RD-19930 35.04 RD-19900 442.16 RD-19810 319.86 RD-19720 207.72 RD-19660 835.77 RD-19585 801.27 RD-19480 480.59 RC-19960 27.25 RC-19825 63.57 RB-19795 -461.74 RB-19705 243.07 RB-19645 72.68 RB-19570 117.64 RB-19465 640.56 RB-19435 475.03 RB-19360 6976.36 RB-19330 -164.41 RA-19945 -441.23 RA-19915 553.29 RA-19885 1254.47 RA-19285 700.56 QJ-19255 1203.69 PW-19240 1253.76 PW-19030 1103.3 PV-18985 434.25 PT-19090 -20.05 PS-19045 3183.77 PS-18970 260.29 PS-18760 -111.14 PR-18880 5596.2 PP-18955 1533.89 PO-19195 -49.69 PO-19180 1263.25 PO-18865 499.32 PO-18850 23.9 PN-18775 192.04 PM-19135 526.76 PM-18940 157.88 PL-18925 -10.4 PK-19075 2077.9 PK-18910 -798.71 PJ-19015 682.68 PJ-18835 442.14 PH-18790 47.89 PG-18895 384.34 PG-18820 226.7 PF-19225 591.29 PF-19165 243.46 PF-19120 -639.02 PC-19000 -184.34 PC-18745 272.68 PB-19210 21.9 PB-19150 263.27 PB-19105 218.16 PB-18805 228.77 PA-19060 225.86 OT-18730 -925.56 ON-18715 -5.53 NZ-18565 402.56 NW-18400 1007.86 NS-18640 -190.77 NS-18505 19.88 NR-18550 -27.71 NP-18700 630.82 NP-18685 514.5 NP-18670 513.15 NP-18325 1209.13 NM-18520 773.98 NM-18445 2751.7 NL-18310 343.8 NK-18490 121.22 NH-18610 760.16 NG-18430 217.09 NG-18355 313.63 NF-18595 420.15 NF-18475 85.92 NF-18385 -1023.09 ND-18460 443.34 ND-18370 348.56 NC-18625 -358.91 NC-18535 1352.02 NC-18415 -2191.57 NC-18340 574.18 NB-18655 117.64 NB-18580 24.79 MZ-17515 785.15 MZ-17335 151.97 MY-18295 -371.71 MY-17380 306.71 MW-18235

#Question:5d

In [0]:
%sql
Select
  Year,
  customer_id,
  round(sum(Total_Profit), 2) as Profit_By_Year_customer_id
from
  orders_analysis.gold.profit_aggregated
group by
  Year,
  customer_id
order by
  Year,
  customer_id desc

Year customer_id Profit_By_Year_customer_id 2014 ZD-21925 25.49 2014 ZC-21910 328.83 2014 YC-21895 833.77 2014 XP-21865 202.31 2014 WB-21850 106.42 2014 VW-21775 -65.32 2014 VT-21700 33.0 2014 VS-21820 -126.25 2014 VP-21760 118.07 2014 VM-21835 6.51 2014 VM-21685 78.23 2014 VG-21805 -82.88 2014 VG-21790 19.54 2014 VF-21715 -155.58 2014 VD-21670 611.56 2014 TZ-21580 132.3 2014 TW-21025 100.14 2014 TT-21220 18.38 2014 TT-21070 121.4 2014 TS-21610 -227.49 2014 TS-21505 87.45 2014 TS-21430 17.14 2014 TS-21370 16.37 2014 TS-21340 -265.38 2014 TS-21205 471.81 2014 TS-21160 -2.0 2014 TR-21325 28.52 2014 TN-21040 21.25 2014 TM-21010 128.83 2014 TH-21550 26.63 2014 TH-21235 -2.93 2014 TH-21115 5.45 2014 TH-21100 -155.27 2014 TG-21640 89.42 2014 TD-20995 9.23 2014 TC-21535 3.36 2014 TC-21475 -16.47 2014 TC-21295 402.54 2014 TC-20980 151.7 2014 TB-21625 83.99 2014 TB-21595 6.75 2014 TB-21520 -125.24 2014 TB-21400 2590.98 2014 TB-21280 200.14 2014 TB-21250 13.8 2014 TB-21190 161.97 2014 TB-21175 6.04 2014 TA-21385 180.52 2014 SZ-20035 6.49 2014 SW-20755 19.72 2014 SW-20275 11.0 2014 SW-20245 -27.2 2014 SV-20785 189.13 2014 SV-20365 -218.93 2014 ST-20530 92.38 2014 SS-20875 -9.15 2014 SS-20590 15.51 2014 SS-20515 22.1 2014 SS-20410 -952.71 2014 SS-20140 -88.04 2014 SR-20740 146.43 2014 SR-20425 57.49 2014 SP-20920 24.22 2014 SP-20860 1.19 2014 SP-20650 72.4 2014 SP-20620 67.46 2014 SP-20545 2.81 2014 SN-20710 32.94 2014 SM-20950 -372.11 2014 SM-20320 -1787.58 2014 SM-20005 42.45 2014 SL-20155 22.25 2014 SJ-20500 57.93 2014 SH-20635 70.2 2014 SH-20395 -252.19 2014 SH-19975 54.47 2014 SG-20890 2.94 2014 SG-20605 274.85 2014 SG-20470 150.92 2014 SF-20965 12.27 2014 SF-20200 58.03 2014 SF-20065 98.59 2014 SE-20110 54.41 2014 SD-20485 122.77 2014 SC-20725 1.35 2014 SC-20695 52.53 2014 SC-20680 31.53 2014 SC-20575 32.39 2014 SC-20380 -3462.2 2014 SC-20305 93.4 2014 SC-20260 96.77 2014 SC-20095 5462.59 2014 SC-20050 1180.52 2014 SC-20020 -341.91 2014 SB-20290 -1272.36 2014 SB-20185 -66.52 2014 SB-20170 -33.39 2014 SA-20830 351.32 2014 RW-19630 30.78 2014 RS-19765 552.0 2014 RS-19420 318.93 2014 RR-19525 543.57 2014 RP-19855 56.53 2014 RP-19390 262.81 2014 RP-19270 97.86 2014 RM-19675 200.12 2014 RM-19375 13.83 2014 RL-19615 42.29 2014 RH-19510 10.96 2014 RH-19495 34.94 2014 RF-19840 16.17 2014 RF-19735 22.65 2014 RF-19345 92.94 2014 RE-19450 -24.87 2014 RE-19405 6.04 2014 RD-19930 -23.29 2014 RD-19900 186.49 2014 RD-19810 7.26 2014 RD-19720 73.68 2014 RD-19585 341.38 2014 RD-19480 109.52 2014 RC-19960 -98.43 2014 RC-19825 6.88 2014 RB-19795 7.49 2014 RB-19645 24.92 2014 RB-19465 336.96 2014 RB-19435 62.52 2014 RB-19330 -90.1 2014 RA-19945 -436.73 2014 RA-19915 526.43 2014 RA-19885 771.14 2014 RA-19285 0.0 2014 QJ-19255 120.04 2014 PW-19240 -86.27 2014 PW-19030 17.47 2014 PV-18985 -4.56 2014 PT-19090 -8.03 2014 PS-19045 623.2 2014 PS-18970 184.98 2014 PS-18760 11.88 2014 PR-18880 2.0 2014 PP-18955 472.39 2014 PO-19195 -65.99 2014 PO-18865 132.18 2014 PO-18850 56.31 2014 PM-18940 21.06 2014 PL-18925 22.2 2014 PK-19075 13.32 2014 PK-18910 9.07 2014 PJ-19015 2.78 2014 PJ-18835 7.95 2014 PG-18895 -70.73 2014 PG-18820 54.45 2014 PF-19165 -407.68 2014 PF-19120 17.68 2014 PC-19000 -11.83 2014 PC-18745 196.69 2014 PB-19210 21.2 2014 PB-19150 -4.7 2014 PB-19105 204.07 2014 PA-19060 45.84 2014 ON-18715 183.11 2014 NZ-18565 248.03 2014 NW-18400 -390.75 2014 NS-18640 11.2 2014 NR-18550 -123.79 2014 NP-18700 285.95 2014 NP-18685 33.64 2014 NP-18325 314.39 2014 NM-18445 2413.85 2014 NL-18310 8.28 2014 NH-18610 33.2 2014 NG-18430 134.82 2014 NG-18355 219.08 2014 NF-18595 69.0 2014 NF-18475 31.87 2014 NF-18385 186.74 2014 ND-18460 45.43 2014 ND-18370 47.61 2014 NC-18625 36.66 2014 NC-18535 728.2 2014 NC-18415 10.04 2014 NC-18340 11.15 2014 NB-18655 87.38 2014 NB-18580 24.79 2014 MZ-17515 784.8 2014 MY-18295 -103.13 2014 MW-18235 1622.55 2014 MW-18220 1194.41 2014 MV-18190 11.9 2014 MV-17485 15.08 2014 MT-18070 -89.7 2014 MS-17980 9.07 2014 MS-17830 10.92 2014 MS-